In [ ]:
!pip install evaluate accelerate datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("smartgmin/orgnalDataEntrnal",split="train")


Generating train split:   0%|          | 0/22971 [00:00<?, ? examples/s]

In [ ]:
from PIL import Image


In [ ]:
print(dataset['image'][0])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x640 at 0x7A4B0C916CE0>


In [ ]:
import os
from tqdm import tqdm

# الدالة لحفظ الصور في مجلدات بناءً على التصنيف
def save_images_to_folders(dataset):
    target_dirs = {
        0: "data/Mild",
        1: "data/Moderate",
        2: "data/No_DR",
        3: "data/Proliferative_DR",
        4: "data/Severe"
    }

    # إنشاء المجلدات إذا لم تكن موجودة
    for directory in target_dirs.values():
        os.makedirs(directory, exist_ok=True)

    # حفظ الصور في المجلدات بناءً على التصنيف مع شريط تقدم
    for idx, (image, label) in enumerate(tqdm(zip(dataset['image'], dataset['label']), total=len(dataset['image']), desc="Saving images")):
        save_path = os.path.join(target_dirs[label], f"image_{idx}.jpg")
        image.save(save_path)

# مثال على استخدام الدالة


In [ ]:
save_images_to_folders(dataset)


Saving images: 100%|██████████| 22971/22971 [00:46<00:00, 492.20it/s]


In [ ]:
!pip install roboflow


In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="ItXgPAZWt0DYyYfbUnic")
project = rf.workspace("roboflow-universe-projects").project("banana-ripeness-classification")
version = project.version(4)
dataset = version.download("folder")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Banana-Ripeness-Classification-4 in folder:: 100%|██████████| 13495/13495 [00:01<00:00, 7778.44it/s]


In [ ]:
import os

len(os.listdir('/content/newdataset/class1'))

315

In [ ]:
!cat /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.001 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.002 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.003 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.004 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.005 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.006 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.007 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.008 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.009 /content/drive/MyDrive/entrnal/DDR_dataset/DDR-dataset.zip.010 > DDR-dataset.zip

In [ ]:
!unzip DDR-dataset.zip

In [ ]:
# قراءة البيانات من الملف وحساب عدد الصور لكل تصنيف
def count_classifications(file_path):
    # إنشاء قاموس لحفظ العد لكل تصنيف
    class_counts = {i: 0 for i in range(6)}

    # فتح الملف وقراءته
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # المرور على كل سطر وحساب التصنيفات
    for line in lines:
        image_name, classification = line.strip().split()
        classification = int(classification)

        # تحديث العد للتصنيف
        if classification in class_counts:
            class_counts[classification] += 1

    return class_counts




In [ ]:
# المسار إلى ملف الـ txt
file_path = '/content/ddr_dataset/DDR-dataset/DR_grading/train.txt'

# حساب عدد الصور لكل تصنيف
class_counts = count_classifications(file_path)

# طباعة النتائج
for classification, count in class_counts.items():
    print(f"التصنيف {classification}: {count} صور")


التصنيف 0: 3133 صور
التصنيف 1: 315 صور
التصنيف 2: 2238 صور
التصنيف 3: 118 صور
التصنيف 4: 456 صور
التصنيف 5: 575 صور
التصنيف 6: 0 صور


In [ ]:
import os
import shutil

# مسار المجلد الرئيسي الذي يحتوي على الصور
# مسار المجلد الرئيسي الذي يحتوي على الصور
image_dir = '/content/ddr_dataset/DDR-dataset/DR_grading/valid'

# مسار ملف التصنيفات
label_file = '/content/ddr_dataset/DDR-dataset/DR_grading/valid.txt'

# مسار المجلد الجديد الذي سيتم تخزين الصور المصنفة فيه
output_dir = 'newdataset2/valid'

# إنشاء مجلدات التصنيفات (من 0 إلى 5)
for i in range(6):
    os.makedirs(os.path.join(output_dir, f'class{i}'), exist_ok=True)

# عداد لتتبع عدد الصور المنسوخة
counter = 0

# قراءة ملف التصنيفات
with open(label_file, 'r') as f:
    for line in f:
        image_name, label = line.strip().split()
        label = int(label)

        # مسار الصورة الأصلية
        src_path = os.path.join(image_dir, image_name)

        # مسار مجلد التصنيف
        dst_dir = os.path.join(output_dir, f'class{label}')

        # نسخ الصورة إلى مجلد التصنيف
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_dir)
            counter += 1
            # طباعة عداد التقدم كل 100 صورة
            if counter % 100 == 0:
                print(f"تم نسخ {counter} صورة.")
        else:
            print(f"الصورة {image_name} غير موجودة في المجلد {image_dir}")

print(f"العملية اكتملت. تم نسخ {counter} صورة.")


In [ ]:
import os
import shutil
from PIL import Image

# مسار المجلد الرئيسي الذي يحتوي على الصور
image_dir = '/content/DDR-dataset/DR_grading/train'

# مسار ملف التصنيفات
label_file = '/content/DDR-dataset/DR_grading/train.txt'

# مسار المجلد الجديد الذي سيتم تخزين الصور المصنفة فيه
output_dir = 'newdataset2/train'

# إنشاء مجلدات التصنيفات (من 0 إلى 5)
for i in range(6):
    os.makedirs(os.path.join(output_dir, f'class{i}'), exist_ok=True)

# عداد لتتبع عدد الصور المنسوخة
counter = 0

# قراءة ملف التصنيفات
with open(label_file, 'r') as f:
    for line in f:
        image_name, label = line.strip().split()
        label = int(label)

        # مسار الصورة الأصلية
        src_path = os.path.join(image_dir, image_name)

        # مسار مجلد التصنيف
        dst_dir = os.path.join(output_dir, f'class{label}')

        # مسار الصورة الجديدة بعد تغيير الحجم
        dst_path = os.path.join(dst_dir, image_name)

        # نسخ الصورة إلى مجلد التصنيف مع تغيير الحجم
        if os.path.exists(src_path):
            # فتح الصورة وتغيير حجمها
            with Image.open(src_path) as img:
                img_resized = img.resize((640, 640))
                img_resized.save(dst_path)

            counter += 1
            # طباعة عداد التقدم كل 100 صورة
            if counter % 100 == 0:
                print(f"تم نسخ {counter} صورة.")
        else:
            print(f"الصورة {image_name} غير موجودة في المجلد {image_dir}")

print(f"العملية اكتملت. تم نسخ {counter} صورة.")


تم نسخ 100 صورة.
تم نسخ 200 صورة.
تم نسخ 300 صورة.
تم نسخ 400 صورة.
تم نسخ 500 صورة.
تم نسخ 600 صورة.
تم نسخ 700 صورة.
تم نسخ 800 صورة.
تم نسخ 900 صورة.
تم نسخ 1000 صورة.
تم نسخ 1100 صورة.
تم نسخ 1200 صورة.
تم نسخ 1300 صورة.
تم نسخ 1400 صورة.
تم نسخ 1500 صورة.
تم نسخ 1600 صورة.
تم نسخ 1700 صورة.
تم نسخ 1800 صورة.


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ItXgPAZWt0DYyYfbUnic")
project = rf.workspace("myworkspace-v6ba0").project("entrnalclassificrion")
version = project.version(1)
dataset = version.download("folder")


In [ ]:
rf = Roboflow(api_key='ItXgPAZWt0DYyYfbUnic')

# get a workspace
workspace = rf.workspace("myworkspace-v6ba0")

# Upload data set to a new/existing project
workspace.upload_dataset(
    "/content/entrnalclassificrion", # This is your dataset path
    "entrnalclassificrion", # This will either create or get a dataset with the given ID
    num_workers=10,

    project_type="single-label-classification",
    batch_name=None,
    num_retries=0
)

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="ItXgPAZWt0DYyYfbUnic")
project = rf.workspace("roboflow-universe-projects").project("banana-ripeness-classification")
version = project.version(4)
dataset = version.download("clip")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Banana-Ripeness-Classification-4 in clip:: 100%|██████████| 13498/13498 [00:01<00:00, 8018.72it/s]


In [ ]:
import os
import shutil
def coppp(sors , des):
  if not os.path.exists(des):
    os.makedirs(des)

  for i in os.listdir(sors):

    folder = os.path.join(sors,i)

    for j in os.listdir(folder):

      path1 = os.path.join(folder,j)
      path2 = f'{des}/{j}'

      shutil.copy(path1,path2)

In [ ]:
coppp('/content/entrnalclassificrion/valid','/content/datasset/valid/images')

In [ ]:
import os

def create_txt_files(input_file_path, output_folder_path):
    # قراءة محتوى الملف test.txt
    with open(input_file_path, 'r') as f:
        lines = f.readlines()

    # التأكد من وجود المجلد الوجهة، وإذا لم يكن موجودًا يتم إنشاؤه
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # المرور على كل سطر في الملف
    for line in lines:
        # تقسيم السطر إلى اسم الصورة والتصنيف
        image_name, classification = line.strip().split()

        # إنشاء ملف txt لكل صورة داخل المجلد المحدد
        output_filename = os.path.join(output_folder_path, f"{image_name.split('.')[0]}.txt")

        # كتابة التصنيف مع القيم الثابتة إلى الملف
        with open(output_filename, 'w') as out_file:
            out_file.write(f"{classification} 0.5 0.5 1 1\n")

    print(f"تم إنشاء الملفات بنجاح في المجلد: {output_folder_path}")

# مثال على كيفية استخدام الدالة


In [ ]:
create_txt_files('/content/DDR-dataset/DR_grading/valid.txt', '/content/datasset/valid/labels')


تم إنشاء الملفات بنجاح في المجلد: /content/datasset/valid/labels
